In [282]:
from scipy.integrate import solve_ivp, quad
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import numpy as np
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['font.serif'] ="Cambria"
mpl.rcParams['font.family'] ="serif"
mpl.rcParams['font.size']="16"

# Thermodynamic parameters

In [324]:
Psat_constants_but = [51.836, -4019.2, -4.5229, 4.8833e-17, 6]
Psat_constants_ace = [193.69, -8036.7, -29.502, 4.3678e-2,  1]
Psat_constants_dee = [136.9,  -6954.3, -19.254, 2.4508e-2,  1]
Psat_constants_etn = [73.304, -7122.3, -7.1424, 2.8853e-6,  2]
Psat_constants_wat = [73.649, -7258.2, -7.3037, 4.16533e-6, 2]

def Psat_fn(Psat_constants, T):
    C1, C2, C3, C4, C5 = Psat_constants
    return np.e**(C1 + C2/T + C3*np.log(T) + C4*T**C5) # Pa
def Psats_fn(T):
    return [Psat_fn(Psat_constants, T) for Psat_constants in [Psat_constants_but, Psat_constants_ace, Psat_constants_dee, Psat_constants_etn, Psat_constants_wat]]
z = Psats_fn(48+273.15)
np.array(z)/1000

array([578.84305124, 245.14279014, 160.18018268,  26.87138091,
        11.17752568])

In [331]:
Cp_v_constants_but = [0.64257e5, 2.0618e5, 1.6768e3, 1.3324e5,  757.06]
Cp_v_constants_ace = [0.4451e5,  1.0687e5, 1.6141e3, 0.6135e5,  737.8]
Cp_v_constants_dee = [0.8621e5,  2.551e5,  1.5413e3, 1.437e5,   688.9]
Cp_v_constants_etn = [0.492e5,   1.4577e5, 1.6628e3, 0.939e5,   744.7]
Cp_v_constants_wat = [0.33363e5, 0.2679e5, 2.6105e3, 0.08896e5, 1169]

Cp_l_constants_but = [182050, -1611,   11.963,    -0.037454, 4.5027e-5]
Cp_l_constants_ace = [115100, -433,    1.425,     0,         0]
Cp_l_constants_dee = [44400,  1301,    -5.5,      0.008763,  0]
Cp_l_constants_etn = [102640, -139.63, -0.030341, 0.0020386, 0]
Cp_l_constants_wat = [276370, -2090.1, 8.125,     -0.014116, 9.3701e-6]


λ_constants_but = [3.3774e7,  0.5107,  -0.17304,  0.05181, 419.5]
λ_constants_ace = [3.8366e7,  0.40081, 0,         0,       466]
λ_constants_dee = [4.06e7   , 0.3868,  0,         0,       466.7]
λ_constants_etn = [5.5789e7,  0.31245, 0,         0,       514]
λ_constants_wat = [5.2053e7,  0.3199,  -0.212,    0.25795, 647.096]

def Cp_v_component_fn(Cp_v_constants, T):
    C1, C2, C3, C4, C5 = Cp_v_constants
    return (C1 + C2*((C3/T)/(np.sinh(C3/T)))**2 + C4*((C5/T)/(np.cosh(C5/T)))**2)*1e-3 # J.mol.K-1

def Cp_l_mix_fn(mol_fracs, T):
    Cps = [Cp_l_component_fn(Cp_l_constants, T) for Cp_l_constants in [Cp_l_constants_but, Cp_l_constants_ace, Cp_l_constants_dee, Cp_l_constants_etn, Cp_l_constants_wat]]
    return np.sum(np.array(mol_fracs)*np.array(Cps)) # J.mol.K-1

def Cp_l_component_fn(Cp_l_constants, T):
    C1, C2, C3, C4, C5 = Cp_l_constants
    return (C1 + C2*T + C3*T**2 + C4*T**3 + C5*T**4)*1e-3 # J.mol.K-1

def λ_component_fn(λ_constants, T): # 2-155
    C1, C2, C3, C4, Tc = λ_constants
    Tr = T/Tc
    return C1*(1 - Tr)**(C2 + C3*Tr + C4*Tr**2)*1e-3 # J.mol-1

def λs_fn(T):
    return [λ_component_fn(λ_constants, T) for λ_constants in [λ_constants_but, λ_constants_ace, λ_constants_dee, λ_constants_etn, λ_constants_wat]]

def hls_fn(T1, T2):
    return [quad(lambda T: Cp_l_component_fn(Cp_l_constants, T), T1, T2)[0] for Cp_l_constants in [Cp_l_constants_but , Cp_l_constants_ace , Cp_l_constants_dee , Cp_l_constants_etn , Cp_l_constants_wat]]

def Hvs_fn(T1, T2):
    return [quad(lambda T: Cp_v_component_fn(Cp_v_constants, T), T1, T2)[0] for Cp_v_constants in [Cp_v_constants_but , Cp_v_constants_ace , Cp_v_constants_dee , Cp_v_constants_etn , Cp_v_constants_wat]]
np.array(λs_fn(48 + 273.15))/1000

array([18.67180327, 24.01864521, 25.86993485, 41.07003074, 43.01156116])

In [285]:
def α_fn(T): # wat as reference
    return Psat_fn(Psat_constants_but, T)/Psat_fn(Psat_constants_wat, T), Psat_fn(Psat_constants_ace, T)/Psat_fn(Psat_constants_wat, T), Psat_fn(Psat_constants_dee, T)/Psat_fn(Psat_constants_wat, T), Psat_fn(Psat_constants_etn, T)/Psat_fn(Psat_constants_wat, T), 1 

def solve_T_bubble_fn(var, *args):
    T = var
    xs, P = args
    return (Psat_fn(Psat_constants_but, T)*xs[0] + Psat_fn(Psat_constants_ace, T)*xs[1] + Psat_fn(Psat_constants_dee, T)*xs[2] + Psat_fn(Psat_constants_etn, T)*xs[3] + Psat_fn(Psat_constants_wat, T)*xs[4] - P)*1e-5

def solve_T_dew_fn(var, *args):
    T = var
    xs, P = args
    return xs[0]*P/Psat_fn(Psat_constants_but, T) + xs[1]*P/Psat_fn(Psat_constants_ace, T) + xs[2]*P/Psat_fn(Psat_constants_dee, T) + xs[3]*P/Psat_fn(Psat_constants_etn, T) + xs[4]*P/Psat_fn(Psat_constants_wat, T) - 1

# FUG simulation

In [286]:
xd_wat = 0.01 # mole frac HK in D
xb_etn = 0.0002/2 # mole frac LK in B

F_but = 0.040180281893826955 # mol.s-1
F_ace = 0.0035030494427545906 # mol.s-1
F_dee = 0.021874672321246194 # mol.s-1
F_etn = 0.0563584110070789 # mol.s-1
F_wat = 0.8711558838955872 # mol.s-1 

Tf = 48 + 273.15
Pf = 1.3*1e5 # Pa
#Pf = 1.3e5

M = np.mean([1.05, 1.25])
Tf

321.15

In [287]:
F_total = sum([F_but, F_ace, F_dee, F_etn, F_wat])
zs = z_but, z_ace, z_dee, z_etn, z_wat = np.array([F_but, F_ace, F_dee, F_etn, F_wat])/F_total
αs = α_fn(Tf)
αab = αs[3]
#αs = [5.31804/0.873626, 3.59671/0.873626, 2.45508/0.873626, 1.72072/0.873626, 1]

In [288]:
MM_etn = 46.068
MM_ace = 44.053
MM_dee = 74.122
MM_wat = 18.015
MM_but = 56.106
MM_list = [MM_but, MM_ace, MM_dee, MM_etn, MM_wat]
zs, F_total*3.6, sum(np.array(MM_list)*np.array(zs))

(array([0.04046058, 0.00352749, 0.02202727, 0.05675157, 0.87723309]),
 3.575060274817778,
 22.475968520962212)

## Mass balance

In [289]:
def solve_D_fn(var, *args):
    D = var
    F_total = args
    return (F_but + F_ace + F_dee)/D + (F_etn - xb_etn*(F_total - D))/D + xd_wat - 1

In [290]:
xb_wat = 1 - xb_etn
D = fsolve(solve_D_fn, [F_total/2], args=(F_total))[0]
B = F_total - D

xb_wat = 1 - xb_etn
xd_but = F_but/D
xd_ace = F_ace/D
xd_dee = F_dee/D
xd_etn = (F_etn - xb_etn*(F_total - D))/D

xds = [xd_but, xd_ace, xd_dee, xd_etn, xd_wat]
xbs = [0, 0, 0, xb_etn, xb_wat]
D, B, xd_etn, xb_wat

(0.1230600135721291, 0.8700122849883647, 0.457268028380297, 0.9999)

In [291]:
xds

[0.32650964945876676,
 0.028466187684120074,
 0.1777561344768161,
 0.457268028380297,
 0.01]

## Feed quality

In [292]:
Tf_bubble = fsolve(solve_T_bubble_fn, 450, args=(zs, Pf))[0]
Tf_bubble, solve_T_bubble_fn(Tf_bubble, zs, Pf)

(359.41930915343613, 1.0331859812140465e-14)

In [293]:
λf = np.sum(np.array(λs_fn(Tf))*zs)
q = (np.sum(np.array(hls_fn(Tf, Tf_bubble))*zs) + λf)/λf
q

1.0779006559759128

## Fenske

In [294]:
Nmin = np.log((xd_etn/xd_wat)/(xb_etn/xb_wat))/np.log(αab)
Nmin

14.858149269993266

In [295]:
Nfmin = np.log((xd_etn/xd_wat)/(z_etn/z_wat))/np.log(αab)
Nfmin

7.479591910704064

## Underwood

In [296]:
ΔVf = F_total*(1 - q)
def solve_φ_fn(φ):
    return np.sum([αs[i]*F_total*zs[i]/(αs[i] - φ) for i in range(len(zs))]) - ΔVf
φ = fsolve(solve_φ_fn, np.mean([αab, 1]))[0]
if (φ > 1) and (φ < αab):
    print('passed')
φ, solve_φ_fn(φ)

passed


(2.1752845513743004, -5.287437154777308e-15)

In [297]:
Vmin = sum([αs[i]*D*xds[i]/(αs[i] - φ) for i in range(len(xds))])
Vmin

0.6619074469649162

In [298]:
Lmin = Vmin - D
rmin = Lmin/D
rmin

4.378736989793624

In [299]:
rmin*M

5.0355475382626675

## Gilliland

In [300]:
abscissa = (M - 1)/(1/rmin + M)
abscissa

0.10882368903652215

In [301]:
#y = 1 - np.exp((1 + 54.4*abscissa)/(11 + 117.2*abscissa)*((abscissa - 1)/np.sqrt(abscissa)))
y = 0.545827 - 0.591422*abscissa + 0.002743/abscissa
abscissa, y

(0.10882368903652215, 0.5066721851560037)

In [302]:
N_total = (-Nmin - y)/(y - 1)
N_total

31.14525674983083

## Kirkbride

In [303]:
rhs = (z_wat/z_etn*(xb_etn/xd_wat)**2*(B/D))**0.26
Nf = (rhs*N_total + 1)/(1 + rhs)
Nf

8.116824026479774

## Duties

In [304]:
Td_bubble = fsolve(solve_T_bubble_fn, 300, args=(xds, Pf))[0]
Td_dew = fsolve(solve_T_dew_fn, 350, args=(xds, Pf))[0]
Td_bubble, Td_dew, solve_T_bubble_fn(Td_bubble, xds, Pf), solve_T_dew_fn(Td_dew, xds, Pf)

(301.0215182981723,
 342.8883477530606,
 -4.365574568510056e-16,
 -4.9960036108132044e-15)

In [313]:
R = rmin*M
L = R*D
V = L + D
λv = np.sum(np.array(λs_fn(Td_dew))*xds)
Qcond = V*λv + quad(lambda T: Cp_l_mix_fn(xds, T), Td_bubble, Td_dew)[0]
Qcond

27355.092489504517

In [ ]:
Tb_dew = fsolve(solve_T_dew_fn, 380, args=([0, 0, 0, 0, 1], Pf))[0]
Tb_dew, solve_T_dew_fn(Tb_dew, [0, 0, 0, 0, 1], Pf)

(380.3092961237204, 5.551115123125783e-15)

In [320]:
L_bar = L + q*F_total
V_bar = L_bar + V - F_total - L

λl = λ_component_fn(λ_constants_wat,Tb_dew)
Qboil = V_bar*λl
Qboil

33179.56736726744

In [281]:
L_bar

1.690107830451569

In [312]:
xds

[0.32650964945876676,
 0.028466187684120074,
 0.1777561344768161,
 0.457268028380297,
 0.01]